# Optimize CBCT parameters through CG with reconstructions

In [7]:
import numpy as np
from scipy.optimize import minimize
from skimage.filters import sobel
from skimage.morphology import disk
from skimage import exposure

In [11]:
import sys
sys.path.append('/Users/kaestner/git/build-imagingsuite/Release/lib')

import muhrectomo as mt
            

ImportError: dlopen(/Users/kaestner/git/build-imagingsuite/Release/lib/muhrectomo.cpython-39-darwin.so, 0x0002): Library not loaded: @rpath/libStdBackProjectors.dylib
  Referenced from: <E849F2D4-C317-366D-B787-8DC6A4643756> /Users/kaestner/git/build-imagingsuite/Release/lib/muhrectomo.cpython-39-darwin.so
  Reason: tried: '/Users/kaestner/git/build-imagingsuite/Release/lib/../Frameworks/libStdBackProjectors.dylib' (no such file), '/Users/kaestner/git/build-imagingsuite/Release/lib/../../../lib/libStdBackProjectors.dylib' (no such file), '/Users/kaestner/git/build-imagingsuite/Release/lib/lib/libStdBackProjectors.dylib' (no such file), '/Users/kaestner/git/build-imagingsuite/Release/lib/libStdBackProjectors.dylib' (no such file), '/Users/kaestner/.conan2/p/pybin6c4510a89d30a/p/lib/libStdBackProjectors.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/kaestner/.conan2/p/pybin6c4510a89d30a/p/lib/libStdBackProjectors.dylib' (no such file), '/Users/kaestner/.conan2/p/b/lib

In [17]:
def reconstruct_slice(SDD, SOD, piercing_point, geometry_params):
    """
    Mock function to reconstruct a slice given the geometry parameters.
    Replace this with actual reconstruction code.
    """
    # Placeholder for reconstruction logic
    # geometry_params should include detector info, pixel pitch, etc.
    reconstructed_slice = np.random.rand(256, 256)  # Dummy 2D image
    return reconstructed_slice

def compute_metrics(reconstructed_slice):
    """
    Compute variance, entropy, total variation, and gradient magnitude.
    """
    # Variance
    variance = np.var(reconstructed_slice)
    
    # Entropy
    # ent = entropy(exposure.rescale_intensity(reconstructed_slice), disk(3))
    # mean_entropy = np.mean(ent)
    
    # Total Variation (TV)
    # tv = denoise_tv_chambolle(reconstructed_slice, weight=0.1, multichannel=False)
    # total_variation = np.sum(np.abs(reconstructed_slice - tv))
    
    # Gradient Magnitude (sharpness)
    gradient_magnitude = np.mean(np.abs(sobel(reconstructed_slice)))
    
    # return variance, mean_entropy, total_variation, gradient_magnitude
    return variance, gradient_magnitude

def cost_function(params, geometry_params):
    """
    Cost function for optimization.
    Combines variance, entropy, total variation, and gradient magnitude with weights.
    """
    SDD, SOD, piercing_point = params
    reconstructed_slice = reconstruct_slice(SDD, SOD, piercing_point, geometry_params)
    # variance, ent, total_variation, gradient_magnitude = compute_metrics(reconstructed_slice)
    variance, gradient_magnitude = compute_metrics(reconstructed_slice)
    
    # Combine metrics into a single cost
    # Adjust weights to emphasize the importance of each metric
    weight_variance = 1.0
    weight_entropy = 1.0
    weight_tv = 0.5
    weight_gradient = 0.5

    cost = (
        - weight_variance * variance  # Maximize variance
        - weight_gradient * gradient_magnitude  # Maximize sharpness
    )
    # cost = (
    #     -weight_variance * variance  # Maximize variance
    #     + weight_entropy * ent       # Minimize entropy
    #     + weight_tv * total_variation  # Minimize total variation (reduce noise)
    #     - weight_gradient * gradient_magnitude  # Maximize sharpness
    # )
    return cost

def optimize_geometry(initial_guess, geometry_params):
    """
    Perform optimization using the conjugate gradient method.
    """
    result = minimize(
        fun=cost_function,
        x0=initial_guess,
        args=(geometry_params,),
        method='CG',
        options={'disp': True}
    )
    return result

# Known inputs
geometry_params = {
    "detector_pixel_pitch": 0.1,  # Example value in mm
    "magnification": 2.0,
    "max_cone_angle": 40,  # degrees
    "max_SDD": 2000.0,  # Example value in mm
    "center_of_rotation": 128.0,  # Example value in pixel units
}

# Initial guesses for SDD, SOD, and piercing_point
initial_guess = [1500.0, 1000.0, 0.0]  # Example initial values

# Run optimization
result = optimize_geometry(initial_guess, geometry_params)

# Optimized parameters
print("Optimized Parameters:")
print("SDD:", result.x[0])
print("SOD:", result.x[1])
print("Vertical Piercing Point:", result.x[2])

         Current function value: -0.195826
         Iterations: 1
         Function evaluations: 90
         Gradient evaluations: 20
Optimized Parameters:
SDD: 1499.9999910272388
SOD: 1000.00001770296
Vertical Piercing Point: 1.111844011758008e-05


/Users/kaestner/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:703: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(fun, x0, args, jac, callback, **options)
